In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import re

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import csv 
pd.options.mode.chained_assignment = None

np.random.seed(31415)
test_file = pd.read_csv("/kaggle/input/titanic/test.csv")
print(test_file.shape) #(418,11)
print(test_file.describe())

train_file = pd.read_csv("/kaggle/input/titanic/train.csv")
print(train_file.shape) #(891,12)
print(train_file.describe())

'''
null_test = train_file['Sex'].isnull().sum()
print("sesso null nel test: ", null_test)
null_test = train_file['SibSp'].isnull().sum()
print("sib null nel test: ", null_test)
null_test = train_file['Fare'].isnull().sum()
print("fare null nel test: ", null_test)
null_test = train_file['Name'].isnull().sum()
print("nome null nel test: ", null_test)
null_test = train_file['Parch'].isnull().sum()
print("parch null nel test: ", null_test)
'''
samples = train_file[['Pclass', 'Name','Sex', 'Age', 'SibSp', 'Fare', 'Ticket', 'Parch','Embarked']] #excluding cabin and passID, many nans
X_test = test_file[['Pclass', 'Name','Sex', 'Age', 'SibSp', 'Fare', 'Ticket','Parch', 'Embarked']]
print(samples.shape)

#Preparing data

samples["Age"].fillna(
    samples.groupby(["Sex"])["Age"].transform(
        lambda x: x.median()
    ), inplace=True)

samples["Fare"].fillna(
    samples.groupby(["Sex"])["Fare"].transform(
        lambda x: x.mean()
    ), inplace=True)

samples["Embarked"].fillna("S", inplace=True)

X_test["Age"].fillna(
    X_test.groupby(["Sex"])["Age"].transform(
        lambda x: x.mean()
    ), inplace=True)

X_test["Fare"].fillna(
    X_test.groupby(["Sex"])["Fare"].transform(
        lambda x: x.mean()
    ), inplace=True)


count = 0
for idx in samples['Name']: #turn name into title
    idx = idx.split(",")[1].split(".")[0]
    samples.iloc[count,1] = idx
    count=count+1

count = 0
for idx in X_test['Name']: #turn name into title, more meaningful
    idx = idx.split(",")[1].split(".")[0]
    X_test.iloc[count,1] = idx
    count=count+1

def substitute_with_length(string):
    if string != '':
        return len(string)
    else:
        return 0

samples['Name'] = samples['Name'].apply(lambda x: substitute_with_length(x))
X_test['Name'] = X_test['Name'].apply(lambda x: substitute_with_length(x))
print(samples['Name'])

X_test["Name"].fillna(
    X_test.groupby(["Sex"])["Name"].transform(
        lambda x: x.mean()
    ), inplace=True)

samples["Name"].fillna(
    samples.groupby(["Sex"])["Name"].transform(
        lambda x: x.mean()
    ), inplace=True)


def remove_non_numeric(string):
    result = 0
    if (len(string)>0):
        result = re.sub('[^0-9]', '',string)
    if (result==''):
        result = 0
    return result

# Apply the function to each row of the column
samples['Ticket'] = samples['Ticket'].apply(lambda x: remove_non_numeric(x))
X_test['Ticket'] = X_test['Ticket'].apply(lambda x: remove_non_numeric(x))

samples = pd.get_dummies(samples, prefix=['Sex', 'Embarked'],
                         columns=['Sex', 'Embarked']) #one-hot encoding
print(samples.shape)
samples_np = np.array(samples)

X_test = pd.get_dummies(X_test, prefix=['Sex', 'Embarked'],
                         columns=['Sex', 'Embarked']) #one-hot encoding
print(X_test.shape)
X_test = np.array(X_test)

labels = train_file[ 'Survived']
labels = np.array(labels)

#random forest classifier model
clf = RandomForestClassifier()

parameters = {
    "n_estimators":[5,10,50,100,250],
    "max_depth":[2,4,8,16,32,None]
}

print(samples_np)
#grid search of best parameters and kfold cross validation (k=10)
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(clf,parameters,cv=10)
cv.fit(samples_np,labels)


def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')
        
display(cv)

best_RF_model = RandomForestClassifier(n_estimators = 250)
best_RF_model.fit(samples_np,labels)
predictions = best_RF_model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_file.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
(418, 11)
       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000    2.265550   30.272590    0.447368    0.392344   35.627188
std     120.810458    0.841838   14.181209    0.896760    0.981429   55.907576
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000   21.000000    0.000000    0.000000    7.895800
50%    1100.500000    3.000000   27.000000    0.000000    0.000000   14.454200
75%    1204.750000    3.000000   39.000000    1.000000    0.000000   31.500000
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200
(891, 12)
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    